Множество / Dataset


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
trening_1=pd.read_csv("/content/drive/MyDrive/dataset_sandra_new_300_emocii.csv")
trening_2=pd.read_csv("/content/drive/MyDrive/Dataset_NLP_elena_300.csv")


Data preparation / Предпроцесирање на податочното множество

In [ ]:
# solving bug / poravanje na greska
trening_2.rename(columns={"Whilst death anxiety itself isn't a disorder, existential fears lie at the core of many anxiety and depressive disorders. This means that it is often linked to these kinds of mental health issues – Generalised Anxiety Disorder (GAD) in particular, which is characterised by frequent and uncontrollable worrying.":"A"},inplace=True)

In [ ]:
trening_2.rename(columns={'A':'Answers'},inplace=True)
trening_2.columns[1]


'Answers'

In [ ]:
trening_1.rename(columns={'Emocia': 'Sentiment'}, inplace=True)
trening_1.rename(columns={"Question":"Questions"},inplace=True)
trening_1.rename(columns={"Answer":"Answers"},inplace=True)
trening_1.head()

,Questions,Answers,Sentiment
0,I need help with abusive neighbor,I know it's not a great help but have you ever...,fear
1,I need help with abusive neighbor,I suppose you could try ignoring it. I mean- i...,faer
2,I need help with abusive neighbor,I understand about not wanting to move- short ...,fear
3,\n\n \nIs the MBTi test accurate and reliable?,"Just a thought, although we know the tests, wh...",curiosity
4,Is the MBTi test accurate and reliable?,So first off I am not a professional so this i...,curiosity


In [ ]:
trening_1.Sentiment.unique()

array(['fear', 'faer', 'curiosity', 'grief', 'happy', 'love'],
      dtype=object)

In [ ]:
trening_1.Sentiment.replace("faer","fear",inplace=True)

In [ ]:
trening_2.Sentiment.unique()

array(['Grief', 'Curiosity', 'Love', 'Fear', 'Happiness', nan],
      dtype=object)

In [ ]:
trening_2.Sentiment.replace("Happiness","happy",inplace=True)
trening_2.Sentiment.replace("Love","love",inplace=True)
trening_2.Sentiment.replace("Curiosity","curiosity",inplace=True)
trening_2.Sentiment.replace("Fear","fear",inplace=True)
trening_2.Sentiment.replace("Grief","grief",inplace=True)




In [ ]:
trening = pd.concat([trening_1, trening_2], ignore_index=True, sort=False)
trening.head()


,Questions,Answers,Sentiment
0,I need help with abusive neighbor,I know it's not a great help but have you ever...,fear
1,I need help with abusive neighbor,I suppose you could try ignoring it. I mean- i...,fear
2,I need help with abusive neighbor,I understand about not wanting to move- short ...,fear
3,\n\n \nIs the MBTi test accurate and reliable?,"Just a thought, although we know the tests, wh...",curiosity
4,Is the MBTi test accurate and reliable?,So first off I am not a professional so this i...,curiosity


In [ ]:
trening.Sentiment.unique()

array(['fear', 'curiosity', 'grief', 'happy', 'love', nan], dtype=object)

In [ ]:
trening['Sentiment'].isnull().values.sum()
trening.Sentiment.replace(np.nan,"love",inplace=True)
trening.Sentiment.unique()

array(['fear', 'curiosity', 'grief', 'happy', 'love'], dtype=object)

In [ ]:
love_answers=[]
happy_answers=[]
fear_answers=[]
curiosity_answers=[]
grief_answers=[]

love_questions=[]
happy_questions=[]
fear_questions=[]
curiosity_questions=[]
grief_questions=[]


In [ ]:
for i in range(0,len(trening_1)):
  if trening.Sentiment[i] == 'love':
    love_answers.append(trening.Answers[i])
    love_questions.append(trening.Questions[i])
  elif trening.Sentiment[i] == 'happy':
    happy_answers.append(trening.Answers[i])
    happy_questions.append(trening.Questions[i])
  elif trening.Sentiment[i] == 'grief':
    grief_answers.append(trening.Answers[i])
    grief_questions.append(trening.Questions[i])
  elif trening.Sentiment[i] == 'curiosity':
    curiosity_answers.append(trening.Answers[i])
    curiosity_questions.append(trening.Questions[i])
  else:
    fear_answers.append(trening.Answers[i])
    fear_questions.append(trening.Questions[i])


Dictionary creating / Креирање на речник

In [ ]:
# used a dictionary to represent an intents JSON file
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "How are you?", "Hi there", "Hi", "Whats up"],
              "responses": ["Hello", "How are you doing?", "Greetings!", "How do you do?"],
             },
             {"tag": "age",
              "patterns": ["how old are you?", "when is your birthday?", "when was you born?"],
              "responses": ["I am machine algorithum you know , I have no age :)"]
             },
             {"tag": "date",
              "patterns": ["what are you doing this weekend?",
"do you want to hang out some time?", "what are your plans for this week"],
              "responses": ["I am available all week", "I don't have any plans", "I am not busy"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "what are you called?", "who are you?"],
              "responses": ["My name is Positiva", "I'm Positiva", "Positiva"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "g2g", "see ya", "adios", "cya"],
              "responses": ["It was nice speaking to you", "See you later", "Speak soon!"]
             },
             {"tag": "love",
              "patterns": love_questions,
              "responses": love_answers
             },
             {"tag": "happy",
              "patterns": happy_questions,
              "responses": happy_answers
             },
             {"tag": "grief",
              "patterns": grief_questions,
              "responses": grief_answers
             },
             {"tag": "curiosity",
              "patterns": curiosity_questions,
              "responses": curiosity_answers
             },
             {"tag": "fear",
              "patterns": fear_questions,
              "responses": fear_answers
             }
]}

Библиотека / Libraries

In [ ]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Preprocesing / Предпроцесирање на множеството

In [ ]:
# initializing lemmatizer to get stem of words / лематизација на зборовите и нивна поделба на зборови, класи == tag и прашања и одговори 
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents 
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# чистење на зборовите од интерпункциски знаци и претварање на сите карактери во мали букви
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
print(words)


["'m", "'re", "'s", "'ve", '3', '3am', '4', 'a', 'abandonment', 'about', 'abused', 'abusive', 'accepted', 'accurate', 'acting', 'addict', 'addicted', 'addiction', 'adios', 'adulthood', 'advice', 'affect', 'afraid', 'after', 'alcohol', 'alcoholic', 'all', 'alone', 'always', 'am', 'an', 'and', 'anger', 'angry', 'anxiety', 'any', 'anymore', 'are', 'at', 'attack', 'attention', 'attitude', 'attractive', 'avoid', 'away', 'bad', 'badly', 'be', 'beat', 'because', 'beer', 'behavior', 'being', 'belong', 'best', 'better', 'between', 'biggest', 'bipolar', 'birthday', 'bitter', 'block', 'blood', 'born', 'boyfriend', 'brain', 'break', 'breakup', 'by', 'bye', 'ca', 'call', 'called', 'can', 'cant', 'cause', 'causing', 'cbd', 'change', 'child', 'childhood', 'clarity', 'coffee', 'cold', 'come', 'concentration', 'considered', 'control', 'controlling', 'cope', 'crowd', 'cry', 'cured', 'cya', 'date', 'day', 'dependence', 'dependent', 'depression', 'detachment', 'diabetes', 'did', 'die', 'diet', 'difference

In [ ]:
print(classes)


['age', 'curiosity', 'date', 'fear', 'goodbye', 'greeting', 'grief', 'happy', 'love', 'name']


In [ ]:
print(doc_X)


['Hello', 'How are you?', 'Hi there', 'Hi', 'Whats up', 'how old are you?', 'when is your birthday?', 'when was you born?', 'what are you doing this weekend?', 'do you want to hang out some time?', 'what are your plans for this week', "what's your name?", 'what are you called?', 'who are you?', 'bye', 'g2g', 'see ya', 'adios', 'cya', 'Family and Friends Dislike Wife.', '\nMy girlfriend acting uninterested and cold lately.', 'I WANT TO FIND A GIRL!', 'How to get my girl attention?', 'How do I get rid of negative thoughts in a relationship?', 'How negativity can kill a relationship?', 'Can distance kill a relationship?', 'What is a toxic relationship?', 'My love make me feel bad about myself.', 'Why do I not get turned on by my boyfriend?', 'Does distance affect love?', 'Is distance a reason to break up?', 'Is it normal to want to be loved?', 'Is it wrong to want to be loved?', 'Can true love die?', 'Why do we need to feel loved?', 'Does true love exist?', 'Why do I want to date someone 

In [ ]:
print(doc_y)


['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 

Neural part preprocessing

In [ ]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model / креирање на т.н вреќа од зборови
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size = 0.2, random_state = 4)



```
# This is formatted as code
```

Neural network (Chatbot)

In [ ]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 400
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=X_train, y=y_train, epochs=200, verbose=1)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 128)               53632     
_________________________________________________________________
dropout_18 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_19 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                650       
Total params: 62,538
Trainable params: 62,538
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
16/16 [==============================] - 0s 2ms/step - loss: 1.7322 - accuracy: 0.3266
Epoch 2/200


Metrics / Метрики

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 0 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])

Test Score:  0.6104627847671509
Test accuracy:  0.9032257795333862


User interaction / Интеракција со корисници

In [ ]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
# running the chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

how to be happy
Get physical. …
be happy
Get A Good Laugh. According to an article on Prevention, a study conducted by Stanford University showed that laughter increases dopamine in our brains, which is a chemical that elevates mood.
can anxiety kill you
The inability to find words can indicate brain injury or infection, strokes, and degenerative diseases like Alzheimer's. However, in those cases, word-forgetting will be only one of many other symptoms. On its own, occasionally forgetting a word is a completely normal part of life.
Can anxiety kill you?
The inability to find words can indicate brain injury or infection, strokes, and degenerative diseases like Alzheimer's. However, in those cases, word-forgetting will be only one of many other symptoms. On its own, occasionally forgetting a word is a completely normal part of life.


KeyboardInterrupt: ignored

Поради while True: е KeyboardInterrupt: